In [3]:
from PIL import Image
import numpy as np
from scipy import signal

In [4]:
img = Image.open('photo.jpg')
I = np.asarray(img).copy() #把img轉成array
I2 = 255-I #反轉色彩
img2 = Image.fromarray(I2)
img2.show()

In [5]:
#低頻濾波: 平滑整個圖片
img = Image.open('photo.jpg')
(w,h) = img.size
img = img.resize((w//10,h//10)) # 縮小
I = np.asarray(img).copy() #把img轉成array
R = I[:,:,0].astype('float') #unsigned int(只能加減)轉float
G = I[:,:,1].astype('float')
B = I[:,:,2].astype('float')
data = ((R+G+B)/3).astype('uint8') #轉灰階
mask = np.array([[1/9,1/9,1/9], #每個人都是周圍(加自己)9個人的平均
                 [1/9,1/9,1/9],
                 [1/9,1/9,1/9]]) 
mask2 = np.array([[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]]) # 只對row平均
data2 = signal.convolve2d(data,mask2,boundary='symm',mode='same') # symm=邊界的會拿另一邊(鏡像的)補，mode為產出的跟原本的大小
img = Image.fromarray(data)
img2 = Image.fromarray(data2)
img.show()
img2.show()

In [6]:
# 高頻濾波: 留附近差最多(變化劇烈)的地方
img = Image.open('photo.jpg')
(w,h) = img.size
#img = img.resize((w//10,h//10)) # 縮小
I = np.asarray(img).copy() #把img轉成array
R = I[:,:,0].astype('float') #unsigned int(只能加減)轉float
G = I[:,:,1].astype('float')
B = I[:,:,2].astype('float')
data = ((R+G+B)/3).astype('uint8') #轉灰階(再轉成影像格式=uint8)
Sx = np.array([[-1,0,1], #x方向的mask(抓x方向變化最劇烈的)
               [-2,0,2],
               [-1,0,1]])
Sy = np.array([[-1,-2,-1], #y方向的mask(抓y方向變化最劇烈的)
               [ 0, 0, 0],
               [ 1, 2, 1]])
Ix = signal.convolve2d(data,Sx,boundary='symm',mode='same')
Iy = signal.convolve2d(data,Sy,boundary='symm',mode='same') 
data2 = (Ix**2)+(Iy**2) #兩邊平方相加(直/橫/斜都處理好了)
data2_1D = np.sort(data2.reshape((-1,1))[:,0]) #轉1D array後sort(為了拿到threshold)
threshold = data2_1D[int(len(data2_1D)*0.8)] #以80%的那個值為threshold
data3 = np.zeros((h,w)).astype('uint8')
data3[data2<threshold]=255
img2 = Image.fromarray(data3)
img2.show()

已知$x,y$(原本圖片的座標)，$p,q$(新圖片要貼的座標)，$z=$dummy variable(要把它消掉) <br>
$\begin{bmatrix}pz\\qz\\z\end{bmatrix} = \begin{bmatrix}h_{11}&h_{12}&h_{13}\\h_{21}&h_{22}&h_{23}\\h_{31}&h_{32}&1\end{bmatrix}\begin{bmatrix}x\\y\\1\end{bmatrix}$ <br>
$xh_{11}+yh_{12}+h_{13}=pz$ <br>
$xh_{21}+yh_{22}+h_{23}=qz$ <br>
$xh_{31}+yh_{32}+1=z$ <br>
$p=\frac{xh_{11}+yh_{12}+h_{13}}{xh_{31}+yh_{32}+1}$ <br> 
$xh_{11}+yh_{12}+h_{13}-pxh_{31}-pyh_{32}=p$ <br>
$q=\frac{xh_{21}+yh_{22}+h_{23}}{xh_{31}+yh_{32}+1}$ <br> 
$xh_{21}+yh_{22}+h_{23}-qxh_{31}-qyh_{32}=q$ <br>
$\begin{bmatrix}
    x & y & 1 & 0 & 0 & 0 & -px & -py\\
    0 & 0 & 0 & x & y & 1 & -qx & -qy\\
    \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots\\
\end{bmatrix}\begin{bmatrix}h_{11}\\\vdots\\h_{32}\end{bmatrix}=\begin{bmatrix}p\\q\\\vdots\end{bmatrix}$ <br>
總共8列(4組座標)=會有4組(x,y)和(p,q)，可解$h_{11}$~$h_{32}$ <br>
$\begin{bmatrix}
    x_1 & y_1 & 1 & 0 & 0 & 0 & -p_1x_1 & -p_1y_1\\
    0 & 0 & 0 & x_1 & y_1 & 1 & -q_1x_1 & -q_1y_1\\
    \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots\\
\end{bmatrix}\begin{bmatrix}h_{11}\\\vdots\\h_{32}\end{bmatrix}=\begin{bmatrix}p_1\\q_1\\\vdots\end{bmatrix}$ <br>

原始照片 -> 要合成的照片的四角 <br>
159,56 -> 0, 0 <br>
388,81 -> 2048, 0 <br>
159, 396 -> 0, 3080 <br>
387, 380 -> 2048, 3080 <br>

In [21]:
x1, y1 = 159, 56
x2, y2 = 388, 81
x3, y3 = 159, 396
x4, y4 = 387, 380

I = Image.open('photo2.jpg')
data = np.asarray(I)

p1, q1 = 0, 0
p2, q2 = data.shape[0], 0
p3, q3 = 0, data.shape[1]
p4, q4 = data.shape[0], data.shape[1]

A = np.zeros((8,8))
b = np.zeros((8,1))
A[0, :] = [x1, y1, 1, 0, 0, 0, -p1*x1, -p1*y1]
A[1, :] = [x2, y2, 1, 0, 0, 0, -p2*x2, -p2*y2]
A[2, :] = [x3, y3, 1, 0, 0, 0, -p3*x3, -p3*y3]
A[3, :] = [x4, y4, 1, 0, 0, 0, -p4*x4, -p4*y4]
A[4, :] = [0, 0, 0, x1, y1, 1, -q1*x1, -q1*y1]
A[5, :] = [0, 0, 0, x2, y2, 1, -q2*x2, -q2*y2]
A[6, :] = [0, 0, 0, x3, y3, 1, -q3*x3, -q3*y3]
A[7, :] = [0, 0, 0, x4, y4, 1, -q4*x4, -q4*y4]
b[:, 0] = [p1, p2, p3, p4, q1, q2, q3, q4]

x = np.linalg.lstsq(A,b,rcond=None)[0] #就是上面那坨

H = np.zeros((3,3)) #h_11到h_32和1
H[0, 0] = x[0]
H[0, 1] = x[1]
H[0, 2] = x[2]
H[1, 0] = x[3]
H[1, 1] = x[4]
H[1, 2] = x[5]
H[2, 0] = x[6]
H[2, 1] = x[7]
H[2, 2] = 1

print(H)

BG = Image.open('BG.jpg')
BG_data = np.asarray(BG).copy()
I = Image.open('photo2.jpg')
data = np.asarray(I)
[BGH,BGW,t] = BG_data.shape
[IH,IW,t] = data.shape
xy1 = np.ones((3,1))
pq1 = np.ones((3,1))
for h in range(BGH):
    for w in range(BGH):
        xy1[0] = w
        xy1[1] = h
        pq1 = np.dot(H,xy1)
        p = int(pq1[0][0]/pq1[2][0])
        q = int(pq1[1][0]/pq1[2][0])
        if(p>=0 and p<IW and q>=0 and q<IH):
            BG_data[h,w,:] = data[q,p,:]
I2 = Image.fromarray(BG_data)
I2.show()
I2.save('109705003_陳以瑄_HW5.jpg')

[[ 7.96588398e-01 -7.26784952e-13 -1.26657555e+02]
 [-6.62893289e-02  6.07210252e-01 -2.34637708e+01]
 [-4.84947303e-04 -1.34626762e-05  1.00000000e+00]]
